# 1DC  Light curve estimation

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.units import Quantity
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from regions import CircleSkyRegion
from astropy.table import unique, Column, Table

import logging

log = logging.getLogger(__name__)

In [ ]:
from gammapy.data import ObservationFilter, DataStore
from gammapy.modeling.models import PowerLawSpectralModel
from gammapy.modeling.models import PointSpatialModel
from gammapy.modeling.models import SkyModel, BackgroundModel
from gammapy.cube import PSFKernel, MapMaker, MapDataset
from gammapy.maps import WcsGeom, MapAxis
from gammapy.irf import make_mean_psf, make_mean_edisp
from gammapy.time import LightCurveEstimator
from gammapy.spectrum import (
    SpectrumExtraction,
    ReflectedRegionsBackgroundEstimator,
)
from gammapy.time import LightCurve, LightCurveEstimator

## Select the data

In [ ]:
CTADATA = "/home/boisson/CTA/PHYS/DC1/1dc/"
data_store = DataStore.from_dir("$CTADATA/index/agn")
data_store.info()
#print()
#print("Number of observations: ", len(data_store.obs_table))
#print('DC1 total observation time (hours): {}'.format(data_store.obs_table['ONTIME'].sum() / 3600.))

In [ ]:
!ls $CTADATA/index/agn

In [ ]:
data_store.obs_table.meta

In [ ]:
# Inputs / parameters
fov_radius = 2 * u.deg
emin, emax = [0.1, 10] * u.TeV 

In [ ]:
# Target definition
target_position = SkyCoord(ra=186.227, dec=21.3796, unit='deg')
on_region_radius = Angle("0.2 deg")   
on_region = CircleSkyRegion(center=target_position, radius=on_region_radius)

In [ ]:
# Get list of relevant observations
table = data_store.obs_table

obs_pos = table.pointing_radec

# Select observations that are at offset 0.2 to 2 deg (fov radius) from the source
mask = target_position.separation(obs_pos) < fov_radius
obs_ids = table["OBS_ID"][mask].data
obs_list = data_store.get_observations(obs_ids)
print(f'Number of observations: {len(obs_ids)}')

In [ ]:
# Look at the first few
table[["OBS_ID", "RA_PNT", "DEC_PNT", "ONTIME", "LIVETIME", 'DEADC', 'TSTART', 'TSTOP', 'DATE-OBS', 'TIME-OBS', 'DATE-END', 'TIME-END', "OBJECT", "IRF"]][mask][:5].show_in_browser(jsviewer=True)

## Perform 1D spectra analysis

In [ ]:
# Estimation of the background, ON - OFF measurement with reflected regions
bkg_estimator = ReflectedRegionsBackgroundEstimator(
    on_region=on_region, observations=obs_list
)
bkg_estimator.run()

In [ ]:
#fig, ax, _ = bkg_estimator.plot()
#ax.legend().set_visible(False)

In [ ]:
energy_axis = MapAxis.from_bounds(
    emin.value, emax.value, 10, unit="TeV", name="energy", interp="log"
)

etrue_axis = MapAxis.from_bounds(
    0.1, 20, 20, unit="TeV", name="energy", interp="log"
)
energy_axis 

In [ ]:
# Note that we are not performing the extraction in time bins
extraction = SpectrumExtraction(
    observations=obs_list,
    bkg_estimate=bkg_estimator.result,
    containment_correction=True,
    e_reco=energy_axis.edges,
    e_true=etrue_axis.edges,
)
extraction.run()
datasets_1d = extraction.spectrum_observations   # -> gammapy.spectrum.dataset.SpectrumDatasetOnOff 

In [ ]:
print(datasets_1d[0])
print(datasets_1d[0].mask_safe)

In [ ]:
extraction.compute_energy_threshold(
    method_lo='area_max',
    area_percent_lo=10,
)
print(datasets_1d[0].mask_safe)

## Light Curve estimation for 1D spectra



In [ ]:
# we need to set the times manually for now
for dataset in datasets_1d:
    dataset.counts.meta = dict()
    dataset.counts.meta["t_start"] = dataset.gti.time_start[0]# time_interval[0]
    dataset.counts.meta["t_stop"]  = dataset.gti.time_stop[-1] #time_interval[1]

In [ ]:
# Define the source model - Use a pointsource + integrated power law model to directly get flux

spectral_model_true = PowerLawSpectralModel(
    index=3.75,
    amplitude=(7.8e-16 * u.Unit('1 / (cm2 s MeV)')).to('1 / (cm2 s TeV)'),
    reference=(200000 * u.MeV).to('TeV'),
)

In [ ]:
for dataset in datasets_1d:
    # Copy the source model
    model = spectral_model_true.copy()
    model.name = "PKSB1222+216"
    dataset.model = model

In [ ]:
lc_maker_1d = LightCurveEstimator(datasets_1d, source="PKSB1222+216", reoptimize=False)

In [ ]:
%%time
lc_1d = lc_maker_1d.run(e_ref=0.2 * u.TeV, e_min=0.1 * u.TeV, e_max=10.0 * u.TeV)

In [ ]:
print(lc_1d.table.colnames)
#lc_1d.table.info()

In [ ]:
lc_1d.table['time_min', 'time_max', 'e_ref', 'e_min', 'e_max', 'ref_dnde', 'ref_flux', 'ref_eflux', 'ref_e2dnde', 'norm', 'loglike', 'norm_err', 'counts', 'norm_errp', 'norm_errn', 'norm_ul', 'sqrt_ts', 'ts', 'norm_scan', 'dloglike_scan', 'flux', 'flux_ul', 'flux_err', 'flux_errp', 'flux_errn'].show_in_browser(jsviewer=True)

In [ ]:
incipit='PKSB1222+216'
filename = 'gammapy_fluxes.ecsv'
lc_1d.table["time_min", "time_max", "flux", "flux_err"].write(filename, format='ascii.ecsv', overwrite=True)

In [ ]:
# Plot gammapy lightcurve
fig, ax = plt.subplots(1, 1, figsize=(15,8))  
ax = lc_1d.plot(marker="o", label="PKSB1222+216")
plt.legend()
ax.semilogy()
ax.set_xlabel('Time (MJD)')
ax.set_ylabel('Flux (cm-2 s-1)')

In [ ]:
lc_1d.compute_fvar()

In [ ]:
filename = 'gammapy_lc_fluxes.png'
print(f'Writing {filename}')
fig.savefig(filename)

### Model light curve

In [ ]:
from gammapy.modeling.models import LightCurveTemplateTemporalModel
#model_lc = LightCurve.read('$CTADATA/models/lightcrv_PKSB1222+216.fits')
path = '$CTADATA/models/lightcrv_PKSB1222+216.fits'
model_lc = LightCurveTemplateTemporalModel.read(path)

In [ ]:
print(model_lc)

In [ ]:
model_lc.table.show_in_browser(jsviewer=True)
#model_lc.table.colnames
#model_lc.table.info()

In [ ]:
model_lc.table.meta

In [ ]:
model_lc_time_ref = model_lc.table.meta['MJDREFI'] + model_lc.table.meta['MJDREFF']                                                                
model_lc_time_ref

In [ ]:
model_lc_normalization = 1e+01 * 6.44664  # scale * value

# The spectrum is constant, so just pre-compute the integral flux, 
# and take the `model_lc_normalization` into account

model_flux = spectral_model_true.integral(emin, emax) * model_lc_normalization
energy =200 *u.GeV
model_dnde = spectral_model_true(energy).to('cm-2 s-1 TeV-1') * model_lc_normalization
print('model_flux:', model_flux)
print('model_dnde:', model_dnde)

In [ ]:
# To compute lc on obs GTI
# compute the mean norm between the start and stop time for each observation 
rows = []
for dataset in datasets_1d:
    # print(dataset.gti.time_stop[-1], dataset.gti.time_start[0])
    # LC method needs float seconds 
    time_min = (dataset.gti.time_start[0] - model_lc_time_ref).mjd *24.*3600
    time_max = (dataset.gti.time_stop[-1] - model_lc_time_ref).mjd *24.*3600.
    norm = model_lc.mean_norm_in_time_interval(time_min, time_max)
    rows.append(dict(
        obs_id=dataset.name,
        time_min=time_min,
        time_max=time_max,
        tstart=dataset.gti.time_start[0].mjd,
        tstop=dataset.gti.time_stop[-1].mjd,
        norm=norm,
    ))
table = Table(rows=rows)
table['flux'] = table['norm'] * model_flux
table['dnde'] = table['norm'] * model_dnde

In [ ]:
#list(Time.FORMATS)

In [ ]:
# Plot model lightcurve
fig, ax = plt.subplots(1, 1, figsize=(15,8))  
ax.plot(table['tstart'], table['flux'], 'o')
ax.semilogy()
ax.set_xlabel('Time (MJD)')
ax.set_ylabel('Flux (cm-2 s-1)')

In [ ]:
incipit='PKSB1222+216'
filename = 'model_fluxes.ecsv'
table.write(filename, format='ascii.ecsv', overwrite=True)

In [ ]:
filename = 'model_lc_fluxes.png'
print(f'Writing {filename}')
fig.savefig(filename)

In [ ]:
table['flux'] / lc_1d.table['flux']